# Data Analysis
## Part1 - Change Concentration

### Fluor

In [1]:
%load_ext autoreload
%autoreload 2

import spectrometer_output_parser
import plotly.io as pio
import plotly.graph_objects as go
import analysis_utils
import numpy as np

pio.renderers.default = 'notebook_connected'

# Consts
PATH_TO_FL_CONCENTRATION_MEASUREMENT = r"G:\.shortcut-targets-by-id\1tk8U4WNz1IW5lUWHDdnNVwRthezya4yZ\Fluorescence\Measurements\Part1\Fe"

CONSENTRATIONS = np.array([0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.001, 0.0008, 0.0005, 0.0001], dtype=np.float64)

In [2]:
fluor_outputs = spectrometer_output_parser.parse_excels_in_folder(PATH_TO_FL_CONCENTRATION_MEASUREMENT)
for output in fluor_outputs:
    scatter = go.Scatter(
        x=output.wavelength_to_intensity_mapping.iloc[:, 0], y=output.wavelength_to_intensity_mapping.iloc[:, 1], mode="markers")
    fig = go.Figure(data=[scatter])
    fig.update_layout(title=output.output_path.stem)
    fig.show()

# out = spectrometer_output_parser.parse_excel_output(r"G:\.shortcut-targets-by-id\1tk8U4WNz1IW5lUWHDdnNVwRthezya4yZ\Fluorescence\Measurements\Part1\Fe\Fl_0.1.ods")
# px.scatter(x=out.wavelength_to_intensity_mapping.iloc[:, 0], y=out.wavelength_to_intensity_mapping.iloc[:, 1])

In [3]:
sorted_fluor_outputs = sorted(fluor_outputs, reverse=True, key=lambda output: output.output_path.stem)

fig_data = []
for output in sorted_fluor_outputs:
    concentration = output.output_path.stem.split("_")[-1]
    dataset = analysis_utils.generate_spectrometer_dataset(output)
    fig_data.append(go.Scatter(
        x=dataset.wavelength, y=dataset.intensity, name=concentration, mode="lines"))

fig = go.Figure(fig_data)
fig.update_layout(legend_title="Concentraion",
                  xaxis_title="Wavelength [nm]", yaxis_title="Intensity [a.u/ms]", height=1000, width=1000, title="Fluorescein Emission Spectrum for Different Concentrations")

fig.show()

In [21]:
integration_results, uncertainties = analysis_utils.integrate_spectrometer_outputs(sorted_fluor_outputs[1:])

integrations_log = np.log(1 / np.abs(integration_results))
propagated_uncertainties = np.abs(1 / integrations_log)*uncertainties

scatter_plot = go.Scatter(x=CONSENTRATIONS[1:], y=integration_results,
                          error_x=dict(array=0.1*CONSENTRATIONS[1:], visible=True), error_y=dict(array=uncertainties, visible=False))
fig = go.Figure([scatter_plot])
fig.show()